<a href="https://colab.research.google.com/github/shantanu2383/FAMAFRENCH_INDUSTRY_FACTORS/blob/main/FAMA_FRENCH_INDUSTRY_FACTORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00. Import Libraries

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

Mounted at /content/gdrive


In [ ]:
#SET PATHS
main="/content/gdrive/MyDrive/FIMA SUMMER 2023/RISK MANAGEMENT/FAMA FRENCH FACTORS/"

# 01. Import Cleaned Returns + Fundamentals data

In [ ]:
clean= main + "_clean/"
returns=pd.read_csv(clean + "pre sort: cleaned returns and fundamentals.csv")

# 02. Import Compustat Data to Map SIC codes to Tickers

In [ ]:
temp= main + "_temp/"
SIC=pd.read_csv(temp + "Compustat2018Through2022.csv")
SIC['date'] = pd.to_datetime(SIC['datadate'])
SIC['Q'] = SIC['date'].dt.to_period('Q')


SIC=SIC[['tic', 'Q', 'sic' ]]
SIC["sic"] = pd.to_numeric(SIC["sic"], errors = 'coerce')



<ipython-input-4-e99ef212e756>:2: DtypeWarning: Columns (17,18,26,30,647,669,674) have mixed types. Specify dtype option on import or set low_memory=False.
  SIC=pd.read_csv(temp + "Compustat2018Through2022.csv")


Map SIC codes to returns data

In [ ]:
returns['date'] = pd.to_datetime(returns['date'])
returns['Q'] = returns['date'].dt.to_period('Q')
SIC = SIC.rename(columns={'tic': 'ticker'})


In [ ]:
unique = SIC[['ticker', 'sic']].drop_duplicates()


In [ ]:
returns = returns.merge(unique, on='ticker', how='left')


# Create SIC code Dictionary

In [ ]:

#Categories 1-5
sic_map = {
    range(100, 200): 'Agriculture',
    range(200, 300): 'Agriculture',
    range(700, 800): 'Agriculture',
    range(910, 920): 'Agriculture',
    range(2048, 2049): 'Agriculture',
    range(2000, 2010): 'Food Products',
    range(2010, 2020): 'Food Products',
    range(2020, 2030): 'Food Products',
    range(2030, 2040): 'Food Products',
    range(2040, 2047): 'Food Products',
    range(2050, 2060): 'Food Products',
    range(2060, 2064): 'Food Products',
    range(2070, 2080): 'Food Products',
    range(2090, 2093): 'Food Products',
    range(2095, 2096): 'Food Products',
    range(2098, 2100): 'Food Products',
    range(2064, 2069): 'Candy & Soda',
    range(2086, 2088): 'Candy & Soda',
    range(2096, 2098): 'Candy & Soda',
    range(2080, 2082): 'Beer & Liquor',
    range(2082, 2083): 'Beer & Liquor',
    range(2083, 2084): 'Beer & Liquor',
    range(2084, 2085): 'Beer & Liquor',
    range(2085, 2086): 'Beer & Liquor',
    range(2100, 2200): 'Tobacco Products'
}

#Categories 6-10
sic_map.update({
    range(920, 1000): 'Toys',
    range(3650, 3653): 'Toys',
    range(3732, 3733): 'Toys',
    range(3930, 3932): 'Toys',
    range(3940, 3950): 'Toys',
    range(7800, 7830): 'Fun',
    range(7830, 7834): 'Fun',
    range(7840, 7842): 'Fun',
    range(7900, 7901): 'Fun',
    range(7910, 7912): 'Fun',
    range(7920, 7930): 'Fun',
    range(7930, 7934): 'Fun',
    range(7940, 7950): 'Fun',
    range(7980, 7981): 'Fun',
    range(7990, 8000): 'Fun',
    range(2700, 2710): 'Books',
    range(2710, 2720): 'Books',
    range(2720, 2730): 'Books',
    range(2730, 2740): 'Books',
    range(2740, 2750): 'Books',
    range(2770, 2772): 'Books',
    range(2780, 2790): 'Books',
    range(2790, 2800): 'Books',
    range(2047, 2048): 'Hshld',
    range(2391, 2393): 'Hshld',
    range(2510, 2520): 'Hshld',
    range(2590, 2600): 'Hshld',
    range(2840, 2844): 'Hshld',
    range(3160, 3162): 'Hshld',
    range(3170, 3172): 'Hshld',
    range(3190, 3200): 'Hshld',
    range(3229, 3230): 'Hshld',
    range(3260, 3261): 'Hshld',
    range(3262, 3264): 'Hshld',
    range(3269, 3270): 'Hshld',
    range(3230, 3232): 'Hshld',
    range(3630, 3640): 'Hshld',
    range(3750, 3752): 'Hshld',
    range(3800, 3801): 'Hshld',
    range(3860, 3862): 'Hshld',
    range(3870, 3874): 'Hshld',
    range(3910, 3912): 'Hshld',
    range(3914, 3915): 'Hshld',
    range(3915, 3916): 'Hshld',
    range(3960, 3963): 'Hshld',
    range(3991, 3992): 'Hshld',
    range(3995, 3996): 'Hshld',
    range(2300, 2391): 'Clths',
    range(3020, 3022): 'Clths',
    range(3100, 3112): 'Clths',
    range(3130, 3132): 'Clths',
    range(3140, 3150): 'Clths',
    range(3150, 3152): 'Clths',
    range(3963, 3966): 'Clths'
})

#Categories 11-15
sic_map.update({
    range(8000, 8100): 'Hlth',
    range(3693, 3694): 'MedEq',
    range(3840, 3850): 'MedEq',
    range(3850, 3852): 'MedEq',
    range(2830, 2831): 'Drugs',
    range(2831, 2832): 'Drugs',
    range(2833, 2834): 'Drugs',
    range(2834, 2835): 'Drugs',
    range(2835, 2836): 'Drugs',
    range(2836, 2837): 'Drugs',
    range(2800, 2810): 'Chems',
    range(2810, 2820): 'Chems',
    range(2820, 2830): 'Chems',
    range(2850, 2860): 'Chems',
    range(2860, 2870): 'Chems',
    range(2870, 2880): 'Chems',
    range(2890, 2900): 'Chems',
    range(3031, 3032): 'Rubbr',
    range(3041, 3042): 'Rubbr',
    range(3050, 3054): 'Rubbr',
    range(3060, 3070): 'Rubbr',
    range(3070, 3080): 'Rubbr',
    range(3080, 3090): 'Rubbr',
    range(3090, 3100): 'Rubbr'
})

#Categories 16-20
sic_map.update({
    range(2200, 2270): 'Txtls',
    range(2270, 2280): 'Txtls',
    range(2280, 2285): 'Txtls',
    range(2290, 2296): 'Txtls',
    range(2297, 2298): 'Txtls',
    range(2298, 2299): 'Txtls',
    range(2393, 2396): 'Txtls',
    range(2397, 2400): 'Txtls',
    range(800, 900): 'BldMt',
    range(2400, 2440): 'BldMt',
    range(2450, 2460): 'BldMt',
    range(2490, 2500): 'BldMt',
    range(2660, 2662): 'BldMt',
    range(2950, 2953): 'BldMt',
    range(3200, 3201): 'BldMt',
    range(3210, 3212): 'BldMt',
    range(3240, 3242): 'BldMt',
    range(3250, 3260): 'BldMt',
    range(3261, 3262): 'BldMt',
    range(3264, 3265): 'BldMt',
    range(3270, 3276): 'BldMt',
    range(3280, 3282): 'BldMt',
    range(3290, 3294): 'BldMt',
    range(3295, 3300): 'BldMt',
    range(3420, 3430): 'BldMt',
    range(3430, 3434): 'BldMt',
    range(3440, 3442): 'BldMt',
    range(3442, 3443): 'BldMt',
    range(3446, 3447): 'BldMt',
    range(3448, 3449): 'BldMt',
    range(3449, 3450): 'BldMt',
    range(3450, 3452): 'BldMt',
    range(3452, 3453): 'BldMt',
    range(3490, 3500): 'BldMt',
    range(3996, 3997): 'BldMt',
    range(1500, 1512): 'Cnstr',
    range(1520, 1530): 'Cnstr',
    range(1530, 1540): 'Cnstr',
    range(1540, 1550): 'Cnstr',
    range(1600, 1700): 'Cnstr',
    range(1700, 1800): 'Cnstr',
    range(3300, 3301): 'Steel',
    range(3310, 3318): 'Steel',
    range(3320, 3326): 'Steel',
    range(3330, 3340): 'Steel',
    range(3340, 3342): 'Steel',
    range(3350, 3358): 'Steel',
    range(3360, 3370): 'Steel',
    range(3370, 3380): 'Steel',
    range(3390, 3400): 'Steel',
    range(3400, 3401): 'FabPr',
    range(3443, 3444): 'FabPr',
    range(3444, 3445): 'FabPr',
    range(3460, 3470): 'FabPr',
    range(3470, 3480): 'FabPr'
})

sic_map.update({
    range(3510, 3520): 'Mach',
    range(3520, 3530): 'Mach',
    range(3530, 3531): 'Mach',
    range(3531, 3532): 'Mach',
    range(3532, 3533): 'Mach',
    range(3533, 3534): 'Mach',
    range(3534, 3535): 'Mach',
    range(3535, 3536): 'Mach',
    range(3536, 3539): 'Mach',
    range(3540, 3550): 'Mach',
    range(3550, 3560): 'Mach',
    range(3560, 3570): 'Mach',
    range(3580, 3581): 'Mach',
    range(3581, 3583): 'Mach',
    range(3585, 3586): 'Mach',
    range(3586, 3587): 'Mach',
    range(3589, 3590): 'Mach',
    range(3590, 3600): 'Mach',
    range(3600, 3601): 'ElcEq',
    range(3610, 3614): 'ElcEq',
    range(3620, 3630): 'ElcEq',
    range(3640, 3645): 'ElcEq',
    range(3645, 3646): 'ElcEq',
    range(3646, 3647): 'ElcEq',
    range(3648, 3650): 'ElcEq',
    range(3660, 3661): 'ElcEq',
    range(3690, 3691): 'ElcEq',
    range(3699, 3700): 'ElcEq',
    range(2296, 2297): 'Autos',
    range(2396, 2397): 'Autos',
    range(3010, 3012): 'Autos',
    range(3537, 3538): 'Autos',
    range(3647, 3648): 'Autos',
    range(3694, 3695): 'Autos',
    range(3700, 3701): 'Autos',
    range(3710, 3712): 'Autos',
    range(3713, 3714): 'Autos',
    range(3714, 3715): 'Autos',
    range(3715, 3716): 'Autos',
    range(3716, 3717): 'Autos',
    range(3792, 3793): 'Autos',
    range(3790, 3792): 'Autos',
    range(3799, 3800): 'Autos',
    range(3720, 3722): 'Aero',
    range(3723, 3725): 'Aero',
    range(3725, 3726): 'Aero',
    range(3728, 3730): 'Aero',
    range(3730, 3732): 'Ships',
    range(3740, 3744): 'Ships'
})

sic_map.update({
    range(3760, 3770): 'Guns',
    range(3795, 3796): 'Guns',
    range(3480, 3490): 'Guns',
    range(1040, 1050): 'Gold',
    range(1000, 1010): 'Mines',
    range(1010, 1020): 'Mines',
    range(1020, 1030): 'Mines',
    range(1030, 1040): 'Mines',
    range(1050, 1060): 'Mines',
    range(1060, 1070): 'Mines',
    range(1070, 1080): 'Mines',
    range(1080, 1090): 'Mines',
    range(1090, 1100): 'Mines',
    range(1400, 1500): 'Mines',
    range(1200, 1300): 'Coal',
    range(1300, 1310): 'Oil',
    range(1310, 1320): 'Oil',
    range(1320, 1340): 'Oil',
    range(1330, 1340): 'Oil',
    range(1370, 1380): 'Oil',
    range(1380, 1382): 'Oil',
    range(1382, 1383): 'Oil',
    range(1389, 1390): 'Oil',
    range(2900, 2913): 'Oil',
    range(2990, 3000): 'Oil'
})


sic_map.update({
    range(4900, 4901): 'Util',
    range(4910, 4912): 'Util',
    range(4920, 4923): 'Util',
    range(4923, 4924): 'Util',
    range(4924, 4926): 'Util',
    range(4930, 4932): 'Util',
    range(4932, 4933): 'Util',
    range(4939, 4940): 'Util',
    range(4940, 4943): 'Util',
    range(4800, 4801): 'Telcm',
    range(4810, 4814): 'Telcm',
    range(4820, 4823): 'Telcm',
    range(4830, 4840): 'Telcm',
    range(4840, 4842): 'Telcm',
    range(4880, 4890): 'Telcm',
    range(4890, 4892): 'Telcm',
    range(4892, 4893): 'Telcm',
    range(4899, 4900): 'Telcm',
    range(7020, 7022): 'PerSv',
    range(7030, 7034): 'PerSv',
    range(7200, 7201): 'PerSv',
    range(7210, 7213): 'PerSv',
    range(7214, 7215): 'PerSv',
    range(7215, 7217): 'PerSv',
    range(7217, 7218): 'PerSv',
    range(7219, 7220): 'PerSv',
    range(7220, 7222): 'PerSv',
    range(7230, 7232): 'PerSv',
    range(7240, 7242): 'PerSv',
    range(7250, 7252): 'PerSv',
    range(7260, 7270): 'PerSv',
    range(7270, 7291): 'PerSv',
    range(7291, 7292): 'PerSv',
    range(7292, 7300): 'PerSv',
    range(7395, 7396): 'PerSv',
    range(7500, 7501): 'PerSv',
    range(7520, 7526): 'PerSv',
    range(7530, 7540): 'PerSv',
    range(7540, 7550): 'PerSv',
    range(7600, 7601): 'PerSv',
    range(7620, 7621): 'PerSv',
    range(7622, 7623): 'PerSv',
    range(7623, 7624): 'PerSv',
    range(7629, 7630): 'PerSv',
    range(7630, 7632): 'PerSv',
    range(7640, 7642): 'PerSv',
    range(7690, 7700): 'PerSv',
    range(8100, 8200): 'PerSv',
    range(8200, 8300): 'PerSv',
    range(8300, 8400): 'PerSv',
    range(8400, 8500): 'PerSv',
    range(8600, 8700): 'PerSv',
    range(8800, 8900): 'PerSv',
    range(7510, 7516): 'PerSv',
    range(2750, 2760): 'BusSv',
    range(3993, 3994): 'BusSv',
    range(7218, 7219): 'BusSv',
    range(7300, 7301): 'BusSv',
    range(7310, 7320): 'BusSv',
    range(7320, 7330): 'BusSv',
    range(7330, 7340): 'BusSv',
    range(7340, 7343): 'BusSv',
    range(7349, 7350): 'BusSv',
    range(7350, 7352): 'BusSv',
    range(7352, 7353): 'BusSv',
    range(7353, 7360): 'BusSv',
    range(7360, 7370): 'BusSv',
    range(7374, 7375): 'BusSv',
    range(7376, 7377): 'BusSv',
    range(7377, 7378): 'BusSv',
    range(7378, 7380): 'BusSv',
    range(7380, 7381): 'BusSv',
    range(7381, 7383): 'BusSv',
    range(7383, 7384): 'BusSv',
    range(7384, 7386): 'BusSv',
    range(7389, 7391): 'BusSv',
    range(7391, 7392): 'BusSv',
    range(7392, 7394): 'BusSv',
    range(7394, 7395): 'BusSv',
    range(7396, 7397): 'BusSv',
    range(7397, 7398): 'BusSv',
    range(7399, 7400): 'BusSv',
    range(7519, 7520): 'BusSv',
    range(8700, 8701): 'BusSv',
    range(8710, 8714): 'BusSv',
    range(8720, 8722): 'BusSv',
    range(8730, 8735): 'BusSv',
    range(8740, 8749): 'BusSv',
    range(8900, 8911): 'BusSv',
    range(8911, 8912): 'BusSv',
    range(8920, 9000): 'BusSv',
    range(4220, 4230): 'BusSv',
    range(3570, 3580): 'Hardw',
    range(3680, 3690): 'Hardw',
    range(3695, 3696): 'Hardw'
})

sic_map.update({
    range(7370, 7373): 'Softw',
    range(7375, 7376): 'Softw',
    range(7373, 7374): 'Softw',
    range(3622, 3623): 'Chips',
    range(3661, 3662): 'Chips',
    range(3662, 3664): 'Chips',
    range(3663, 3664): 'Chips',
    range(3664, 3665): 'Chips',
    range(3665, 3666): 'Chips',
    range(3666, 3667): 'Chips',
    range(3669, 3670): 'Chips',
    range(3670, 3680): 'Chips',
    range(3810, 3811): 'Chips',
    range(3812, 3813): 'Chips',
    range(3811, 3812): 'LabEq',
    range(3820, 3821): 'LabEq',
    range(3821, 3822): 'LabEq',
    range(3822, 3823): 'LabEq',
    range(3823, 3824): 'LabEq',
    range(3824, 3825): 'LabEq',
    range(3825, 3826): 'LabEq',
    range(3826, 3827): 'LabEq',
    range(3827, 3828): 'LabEq',
    range(3829, 3830): 'LabEq',
    range(3830, 3840): 'LabEq',
    range(2520, 2550): 'Paper',
    range(2600, 2640): 'Paper',
    range(2670, 2700): 'Paper',
    range(2760, 2762): 'Paper',
    range(3950, 3956): 'Paper',
    range(2440, 2450): 'Boxes',
    range(2640, 2660): 'Boxes',
    range(3220, 3222): 'Boxes',
    range(3410, 3413): 'Boxes'
})

sic_map.update({
    range(4000, 4014): 'Trans',
    range(4040, 4050): 'Trans',
    range(4100, 4101): 'Trans',
    range(4110, 4120): 'Trans',
    range(4120, 4122): 'Trans',
    range(4130, 4132): 'Trans',
    range(4140, 4143): 'Trans',
    range(4150, 4152): 'Trans',
    range(4170, 4174): 'Trans',
    range(4190, 4200): 'Trans',
    range(4200, 4201): 'Trans',
    range(4210, 4220): 'Trans',
    range(4230, 4232): 'Trans',
    range(4240, 4250): 'Trans',
    range(4400, 4500): 'Trans',
    range(4500, 4600): 'Trans',
    range(4600, 4700): 'Trans',
    range(4700, 4701): 'Trans',
    range(4710, 4713): 'Trans',
    range(4720, 4730): 'Trans',
    range(4730, 4740): 'Trans',
    range(4740, 4750): 'Trans',
    range(4780, 4781): 'Trans',
    range(4782, 4783): 'Trans',
    range(4783, 4784): 'Trans',
    range(4784, 4785): 'Trans',
    range(4785, 4790): 'Trans',
    range(4790, 4791): 'Trans',
    range(4791, 4800): 'Trans',
    range(5000, 5001): 'Whlsl',
    range(5010, 5016): 'Whlsl',
    range(5020, 5024): 'Whlsl',
    range(5030, 5040): 'Whlsl',
    range(5040, 5043): 'Whlsl',
    range(5043, 5044): 'Whlsl',
    range(5044, 5045): 'Whlsl',
    range(5045, 5046): 'Whlsl',
    range(5046, 5047): 'Whlsl',
    range(5047, 5048): 'Whlsl',
    range(5048, 5049): 'Whlsl',
    range(5049, 5050): 'Whlsl',
    range(5050, 5060): 'Whlsl',
    range(5060, 5061): 'Whlsl',
    range(5063, 5064): 'Whlsl',
    range(5064, 5066): 'Whlsl',
    range(5065, 5066): 'Whlsl',
    range(5070, 5079): 'Whlsl',
    range(5080, 5081): 'Whlsl',
    range(5081, 5082): 'Whlsl',
    range(5082, 5084): 'Whlsl',
    range(5084, 5085): 'Whlsl',
    range(5085, 5086): 'Whlsl',
    range(5086, 5088): 'Whlsl',
    range(5088, 5091): 'Whlsl',
    range(5091, 5093): 'Whlsl',
    range(5093, 5094): 'Whlsl',
    range(5094, 5095): 'Whlsl',
    range(5095, 5096): 'Whlsl',
    range(5099, 5100): 'Whlsl',
    range(5100, 5101): 'Whlsl',
    range(5110, 5114): 'Whlsl',
    range(5120, 5123): 'Whlsl',
    range(5130, 5140): 'Whlsl',
    range(5140, 5150): 'Whlsl',
    range(5150, 5160): 'Whlsl',
    range(5160, 5170): 'Whlsl',
    range(5170, 5173): 'Whlsl',
    range(5180, 5183): 'Whlsl',
    range(5190, 5200): 'Whlsl',
    range(5200, 5201): 'Rtail',
    range(5210, 5220): 'Rtail',
    range(5220, 5232): 'Rtail',
    range(5230, 5232): 'Rtail',
    range(5250, 5252): 'Rtail',
    range(5260, 5262): 'Rtail',
    range(5270, 5272): 'Rtail',
    range(5300, 5301): 'Rtail',
    range(5310, 5312): 'Rtail',
    range(5320, 5321): 'Rtail',
    range(5330, 5332): 'Rtail',
    range(5334, 5335): 'Rtail',
    range(5340, 5350): 'Rtail',
    range(5390, 5400): 'Rtail',
    range(5400, 5401): 'Rtail',
    range(5410, 5412): 'Rtail',
    range(5420, 5430): 'Rtail',
    range(5430, 5440): 'Rtail',
    range(5440, 5450): 'Rtail',
    range(5450, 5460): 'Rtail',
    range(5460, 5470): 'Rtail',
    range(5490, 5500): 'Rtail',
    range(5500, 5501): 'Rtail',
    range(5510, 5530): 'Rtail',
    range(5530, 5540): 'Rtail',
    range(5540, 5550): 'Rtail',
    range(5550, 5570): 'Rtail',
    range(5570, 5580): 'Rtail',
    range(5590, 5600): 'Rtail',
    range(5600, 5700): 'Rtail',
    range(5700, 5701): 'Rtail',
    range(5710, 5720): 'Rtail',
    range(5720, 5723): 'Rtail',
    range(5730, 5734): 'Rtail',
    range(5734, 5735): 'Rtail',
    range(5735, 5736): 'Rtail',
    range(5736, 5737): 'Rtail',
    range(5750, 5800): 'Rtail',
    range(5800, 5820): 'Meals',
    range(5820, 5830): 'Meals',
    range(5890, 5900): 'Meals',
    range(7000, 7001): 'Meals',
    range(7010, 7020): 'Meals',
    range(7040, 7050): 'Meals',
    range(7213, 7214): 'Meals',
    range(6000, 6001): 'Banks',
    range(6010, 6020): 'Banks',
    range(6020, 6021): 'Banks',
    range(6021, 6023): 'Banks',
    range(6023, 6025): 'Banks',
    range(6025, 6026): 'Banks',
    range(6026, 6027): 'Banks',
    range(6027, 6028): 'Banks',
    range(6028, 6030): 'Banks',
    range(6030, 6037): 'Banks',
    range(6040, 6060): 'Banks',
    range(6060, 6063): 'Banks',
    range(6080, 6083): 'Banks',
    range(6090, 6100): 'Banks',
    range(6100, 6101): 'Banks',
    range(6110, 6112): 'Banks',
    range(6112, 6114): 'Banks',
    range(6120, 6130): 'Banks',
    range(6130, 6140): 'Banks',
    range(6140, 6150): 'Banks',
    range(6150, 6160): 'Banks',
    range(6160, 6170): 'Banks',
    range(6190, 6200): 'Banks'
})

sic_map.update({
    range(6300, 6301): 'Insur',
    range(6310, 6320): 'Insur',
    range(6320, 6330): 'Insur',
    range(6330, 6332): 'Insur',
    range(6350, 6352): 'Insur',
    range(6360, 6362): 'Insur',
    range(6370, 6380): 'Insur',
    range(6390, 6400): 'Insur',
    range(6400, 6412): 'Insur',
    range(6500, 6501): 'RlEst',
    range(6510, 6511): 'RlEst',
    range(6512, 6513): 'RlEst',
    range(6513, 6514): 'RlEst',
    range(6514, 6515): 'RlEst',
    range(6515, 6516): 'RlEst',
    range(6517, 6520): 'RlEst',
    range(6520, 6530): 'RlEst',
    range(6530, 6532): 'RlEst',
    range(6532, 6533): 'RlEst',
    range(6540, 6542): 'RlEst',
    range(6550, 6554): 'RlEst',
    range(6590, 6600): 'RlEst',
    range(6610, 6612): 'RlEst',
    range(6200, 6300): 'Fin',
    range(6700, 6701): 'Fin',
    range(6710, 6720): 'Fin',
    range(6720, 6723): 'Fin',
    range(6723, 6724): 'Fin',
    range(6724, 6726): 'Fin',
    range(6725, 6726): 'Fin',
    range(6726, 6727): 'Fin',
    range(6730, 6734): 'Fin',
    range(6740, 6780): 'Fin',
    range(6790, 6792): 'Fin',
    range(6792, 6793): 'Fin',
    range(6793, 6794): 'Fin',
    range(6794, 6795): 'Fin',
    range(6795, 6796): 'Fin',
    range(6798, 6799): 'Fin',
    range(6799, 6800): 'Fin',
    range(4950, 4960): 'Other',
    range(4960, 4962): 'Other',
    range(4970, 4972): 'Other',
    range(4990, 4992): 'Other',
})


In [ ]:
def map_sic_to_label(sic):
    for range_, label in sic_map.items():
        if sic in range_:
            return label
    return 'Unknown'

returns['Label'] = returns['sic'].apply(map_sic_to_label)

In [ ]:
returns = returns.drop(['Unnamed: 0', 'level_0', 'index'], axis=1)


In [ ]:
import numpy as np
returns = returns.dropna(subset=['ret', 'mkt_cap'])

value_weighted_returns = returns.groupby(['Label', 'date']).apply(
    lambda x: np.average(x['ret'], weights=x['mkt_cap'])
).to_frame('value_weighted_return')


# Compare with Original Fama French Industry Portfolios

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets



In [ ]:
datasets = get_available_datasets()
datasets

In [ ]:
FF_IND=[dataset for dataset in datasets if '49_Industry_Portfolios_daily' in dataset ]
FF_IND=web.DataReader(FF_IND[0],'famafrench',start='2017-11-01',end='2023-07-01')[0]


In [ ]:
FF_IND

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-01,-0.11,0.18,-0.35,-0.53,-0.16,-2.30,0.52,-0.85,1.03,-0.07,...,0.24,0.21,-0.26,0.20,-0.18,0.19,0.35,0.22,-0.26,-0.20
2017-11-02,-0.54,-0.70,-0.02,-0.27,-1.06,-1.50,0.18,0.07,-0.63,-0.42,...,-1.35,0.35,-0.22,-0.13,0.72,0.67,0.98,-0.94,1.71,-0.10
2017-11-03,-0.36,-0.41,0.22,0.24,-0.07,0.05,0.94,-0.29,-0.34,-0.03,...,0.11,-0.45,0.14,0.72,0.82,-0.11,-0.05,-0.74,-0.62,-0.31
2017-11-06,-0.32,-1.49,-0.94,-0.68,-0.60,-0.03,0.67,0.59,-0.40,0.24,...,-1.09,-0.19,-0.99,-0.24,0.55,-0.15,0.30,0.86,-0.01,-0.08
2017-11-07,0.00,1.12,0.94,0.79,1.07,-0.81,-0.63,-0.71,0.64,-1.26,...,-0.76,-0.40,-0.71,0.01,-0.01,-1.84,-0.19,-0.01,-1.01,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-24,-1.33,-0.46,-0.77,-0.90,-0.39,0.11,1.06,-1.11,-0.70,-1.14,...,-1.57,-1.67,-0.58,0.21,-0.27,-1.39,-0.81,-1.46,-1.19,-0.91
2023-05-25,-0.63,-0.57,-0.67,-0.60,-1.33,-1.87,-0.95,-0.49,-0.31,-0.42,...,-0.62,1.16,0.24,-1.26,-0.10,0.03,-0.61,0.04,1.14,-0.30
2023-05-26,0.51,-0.33,-0.15,-0.02,0.01,0.41,2.78,1.25,0.20,0.48,...,0.65,0.26,0.36,2.40,0.24,1.09,-0.01,0.59,1.13,0.54


In [ ]:
FF_IND.reset_index(inplace=True)
FF_IND['Date']=pd.to_datetime(FF_IND['Date'])

In [ ]:
FF_IND.columns

Index(['Date', 'Agric', 'Food ', 'Soda ', 'Beer ', 'Smoke', 'Toys ', 'Fun  ',
       'Books', 'Hshld', 'Clths', 'Hlth ', 'MedEq', 'Drugs', 'Chems', 'Rubbr',
       'Txtls', 'BldMt', 'Cnstr', 'Steel', 'FabPr', 'Mach ', 'ElcEq', 'Autos',
       'Aero ', 'Ships', 'Guns ', 'Gold ', 'Mines', 'Coal ', 'Oil  ', 'Util ',
       'Telcm', 'PerSv', 'BusSv', 'Hardw', 'Softw', 'Chips', 'LabEq', 'Paper',
       'Boxes', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Banks', 'Insur', 'RlEst',
       'Fin  ', 'Other'],
      dtype='object')

In [ ]:
value_weighted_returns.reset_index(inplace=True)

In [ ]:
unique_labels = value_weighted_returns['Label'].unique()
print(unique_labels)

['Aero' 'Agriculture' 'Autos' 'Banks' 'Beer & Liquor' 'BldMt' 'Books'
 'Boxes' 'BusSv' 'Candy & Soda' 'Chems' 'Chips' 'Clths' 'Cnstr' 'Coal'
 'Drugs' 'ElcEq' 'FabPr' 'Fin' 'Food Products' 'Fun' 'Gold' 'Guns' 'Hardw'
 'Hlth' 'Hshld' 'Insur' 'LabEq' 'Mach' 'Meals' 'MedEq' 'Mines' 'Oil'
 'Other' 'Paper' 'PerSv' 'RlEst' 'Rtail' 'Rubbr' 'Ships' 'Softw' 'Steel'
 'Telcm' 'Tobacco Products' 'Toys' 'Trans' 'Txtls' 'Unknown' 'Util'
 'Whlsl']


In [ ]:
Aero=value_weighted_returns[value_weighted_returns['Label']=="Fun"]

In [ ]:
Aero

In [ ]:
value_weighted_returns

In [ ]:
Aero_FF=FF_IND[['Date', 'Fun  ']]

In [ ]:
FF_IND.columns

In [ ]:
Aero.rename(columns={'date':'Date'}, inplace=True)

<ipython-input-35-eb4dcaac67a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Aero.rename(columns={'date':'Date'}, inplace=True)


In [ ]:
merged_df = Aero.merge(Aero_FF, on='Date', how='left')


In [ ]:
merged_df.corr()

<ipython-input-37-5805580b1846>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df.corr()


,value_weighted_return,Fun
value_weighted_return,1.000000,0.992782
Fun,0.992782,1.000000


In [ ]:



industries=['Aero', 'Agriculture', 'Autos', 'Banks', 'Beer & Liquor', 'BldMt', 'Books', 'Boxes', 'Food Products', 'BusSv', 'Candy & Soda', 'Tobacco Products', 'Toys', 'Fun', 'Chems',
           'Chips', 'Hshld', 'Clths',  'Hlth', 'Cnstr', 'MedEq', 'Coal', 'Drugs', 'Rubbr', 'ElcEq', 'Txtls', 'FabPr', 'Fin', 'Gold', 'Guns', 'Hardw', 'Insur',  'LabEq', 'Steel',  'Mach',  'Meals', 'Mines',  'Oil',  'Other', 'Paper', 'PerSv',
            'RlEst', 'Rtail',  'Ships',  'Softw',  'Telcm', 'Trans', 'Util',  'Whlsl'  ]

industries_ff=['Aero ', 'Agric', 'Autos', 'Banks', 'Beer ', 'BldMt', 'Books', 'Boxes', 'Food ', 'BusSv', 'Soda ', 'Smoke', 'Toys ', 'Fun  ', 'Chems', 'Chips',
               'Hshld', 'Clths', 'Hlth ', 'Cnstr', 'MedEq', 'Coal ', 'Drugs', 'Rubbr', 'ElcEq', 'Txtls', 'FabPr', 'Fin  ', 'Gold ', 'Guns ', 'Hardw', 'Insur',  'LabEq', 'Steel', 'Mach ', 'Meals', 'Mines', 'Oil  ', 'Other', 'Paper', 'PerSv',
               'RlEst', 'Rtail', 'Ships',  'Softw', 'Telcm', 'Trans', 'Util ', 'Whlsl'  ]


for original, new in zip(industries, industries_ff):
    print(f"1: {original}  --> 2: {new}")

1: Aero  --> 2: Aero 
1: Agriculture  --> 2: Agric
1: Autos  --> 2: Autos
1: Banks  --> 2: Banks
1: Beer & Liquor  --> 2: Beer 
1: BldMt  --> 2: BldMt
1: Books  --> 2: Books
1: Boxes  --> 2: Boxes
1: Food Products  --> 2: Food 
1: BusSv  --> 2: BusSv
1: Candy & Soda  --> 2: Soda 
1: Tobacco Products  --> 2: Smoke
1: Toys  --> 2: Toys 
1: Fun  --> 2: Fun  
1: Chems  --> 2: Chems
1: Chips  --> 2: Chips
1: Hshld  --> 2: Hshld
1: Clths  --> 2: Clths
1: Hlth  --> 2: Hlth 
1: Cnstr  --> 2: Cnstr
1: MedEq  --> 2: MedEq
1: Coal  --> 2: Coal 
1: Drugs  --> 2: Drugs
1: Rubbr  --> 2: Rubbr
1: ElcEq  --> 2: ElcEq
1: Txtls  --> 2: Txtls
1: FabPr  --> 2: FabPr
1: Fin  --> 2: Fin  
1: Gold  --> 2: Gold 
1: Guns  --> 2: Guns 
1: Hardw  --> 2: Hardw
1: Insur  --> 2: Insur
1: LabEq  --> 2: LabEq
1: Steel  --> 2: Steel
1: Mach  --> 2: Mach 
1: Meals  --> 2: Meals
1: Mines  --> 2: Mines
1: Oil  --> 2: Oil  
1: Other  --> 2: Other
1: Paper  --> 2: Paper
1: PerSv  --> 2: PerSv
1: RlEst  --> 2: RlEst
1: Rtai

In [ ]:
for x, y in zip(industries, industries_ff):
  temp=value_weighted_returns[value_weighted_returns['Label']==x]
  temp_FF=FF_IND[['Date', y]]
  temp.rename(columns={'date':'Date'}, inplace=True)
  merged_df = temp.merge(temp_FF, on='Date', how='left')
  print(merged_df.corr())




<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(merged_df.corr())
<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only 

                       value_weighted_return     Aero 
value_weighted_return               1.000000  0.979388
Aero                                0.979388  1.000000
                       value_weighted_return     Agric
value_weighted_return               1.000000  0.939779
Agric                               0.939779  1.000000
                       value_weighted_return     Autos
value_weighted_return               1.000000  0.987959
Autos                               0.987959  1.000000
                       value_weighted_return     Banks
value_weighted_return               1.000000  0.800237
Banks                               0.800237  1.000000
                       value_weighted_return   Beer 
value_weighted_return                 1.0000  0.9731
Beer                                  0.9731  1.0000
                       value_weighted_return     BldMt
value_weighted_return               1.000000  0.989756
BldMt                               0.989756  1.000000
                

<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(merged_df.corr())
<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only 

                       value_weighted_return     Hshld
value_weighted_return               1.000000  0.985524
Hshld                               0.985524  1.000000
                       value_weighted_return     Clths
value_weighted_return               1.000000  0.897377
Clths                               0.897377  1.000000
                       value_weighted_return     Hlth 
value_weighted_return               1.000000  0.948603
Hlth                                0.948603  1.000000
                       value_weighted_return     Cnstr
value_weighted_return               1.000000  0.985611
Cnstr                               0.985611  1.000000
                       value_weighted_return     MedEq
value_weighted_return               1.000000  0.989075
MedEq                               0.989075  1.000000
                       value_weighted_return    Coal 
value_weighted_return                1.00000  0.99351
Coal                                 0.99351  1.00000
             

<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(merged_df.corr())
<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(merged_df.corr())
<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

                       value_weighted_return     LabEq
value_weighted_return               1.000000  0.996731
LabEq                               0.996731  1.000000
                       value_weighted_return     Steel
value_weighted_return               1.000000  0.994941
Steel                               0.994941  1.000000
                       value_weighted_return     Mach 
value_weighted_return               1.000000  0.986483
Mach                                0.986483  1.000000
                       value_weighted_return     Meals
value_weighted_return               1.000000  0.988208
Meals                               0.988208  1.000000
                       value_weighted_return     Mines
value_weighted_return               1.000000  0.973057
Mines                               0.973057  1.000000
                       value_weighted_return     Oil  
value_weighted_return               1.000000  0.978919
Oil                                 0.978919  1.000000
          

<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(merged_df.corr())
<ipython-input-42-3c2998c768da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.rename(columns={'date':'Date'}, inplace=True)
<ipython-input-42-3c2998c768da>:6: FutureWarning: The default value of numeric_only 